# pls.is Complex Object Examples

This notebook exercises `pls.is` in deterministic mode (`explain=False`) across a range of Python objects, from primitives to visualization objects.


In [ ]:
from pprint import pprint

import pretty_little_summary as pls


class Address:
    def __init__(self, street, city, zip_code):
        self.street = street
        self.city = city
        self.zip_code = zip_code


class Person:
    def __init__(self, name, age, address, hobbies):
        self.name = name
        self.age = age
        self.address = address
        self.hobbies = hobbies


class OrderItem:
    def __init__(self, sku, quantity, unit_price):
        self.sku = sku
        self.quantity = quantity
        self.unit_price = unit_price

    def total(self):
        return self.quantity * self.unit_price


class PurchaseOrder:
    def __init__(self, order_id, customer, items):
        self.order_id = order_id
        self.customer = customer
        self.items = items

    def total(self):
        return sum(item.total() for item in self.items)


def inspect_obj(label, obj, explain=False):
    """Run pls.is on the object and print the structured output."""

    print(f"=== {label} ===")
    result = pls.describe(obj, explain=explain)
    print(result)
    print("content:")
    print(result.content)
    print("meta:")
    pprint(result.meta)
    if result.history:
        print("history:")
        for line in result.history:
            print(line)
    print()


def add_example(label, builder, examples):
    """Add an example object, skipping if dependencies are missing."""

    try:
        payload = builder()
    except ImportError as err:
        missing = getattr(err, "name", str(err))
        print(f"Skipping {label} (missing dependency: {missing})")
        return
    except Exception as err:  # pragma: no cover - demonstration notebook
        print(f"Skipping {label} due to error: {err}")
        return

    if isinstance(payload, tuple) and len(payload) == 2 and callable(payload[1]):
        obj, cleanup = payload
    else:
        obj, cleanup = payload, None

    examples.append((label, obj, cleanup))


In [ ]:
examples = []

home_address = Address("123 Maple St", "Springfield", "54321")
office_address = Address("500 Elm St", "Metropolis", "60606")

alice = Person("Alice Johnson", 31, home_address, ["cycling", "piano"])
bob = Person("Bob Smith", 28, office_address, ["photography", "bouldering"])
carol = Person("Carol Rivera", 35, office_address, ["gardening", "sourdough baking"])

purchase_order = PurchaseOrder(
    "PO-1001",
    alice,
    [
        OrderItem("SKU-001", 2, 499.0),
        OrderItem("SKU-002", 4, 89.0),
        OrderItem("SKU-003", 1, 2599.0),
    ],
)

company_structure = {
    "name": "Initech Analytics",
    "hq": office_address,
    "departments": [
        {
            "name": "Engineering",
            "lead": alice,
            "projects": [
                {"name": "Phoenix", "owner": alice, "budget": 1_200_000, "status": "active"},
                {"name": "Atlas", "owner": bob, "budget": 850_000, "status": "maintenance"},
            ],
        },
        {
            "name": "Operations",
            "lead": carol,
            "open_orders": [purchase_order],
            "metrics": {"nps": 67, "tickets_open": 14},
        },
    ],
}

number_list = [1, 2, 3, 5, 8, 13]
homogeneous_people = [alice, bob, carol]
heterogeneous_items = [
    alice,
    {"project": "Phoenix", "team_size": 6},
    purchase_order,
    ["dashboard", "pipeline", "etl"],
    3.14159,
]

examples.append(("Integer literal", 42, None))
examples.append(("Custom Person instance", alice, None))
examples.append(("Nested structure with objects", company_structure, None))
examples.append(("List of integers", number_list, None))
examples.append(("List of Person instances", homogeneous_people, None))
examples.append(("Mixed list with objects", heterogeneous_items, None))
examples.append(("PurchaseOrder object", purchase_order, None))

sales_records = [
    {"region": "North", "quarter": "Q1", "revenue": 125_000, "growth": 0.12},
    {"region": "North", "quarter": "Q2", "revenue": 138_500, "growth": 0.10},
    {"region": "South", "quarter": "Q1", "revenue": 98_000, "growth": 0.08},
    {"region": "South", "quarter": "Q2", "revenue": 112_750, "growth": 0.15},
]


def build_numpy_array():
    import numpy as np

    grid = np.linspace(-1.0, 1.0, num=9).reshape(3, 3)
    return grid


def build_pandas_dataframe():
    import pandas as pd

    return pd.DataFrame(sales_records)


def build_polars_dataframe():
    import polars as pl

    return pl.DataFrame(sales_records)


def build_matplotlib_figure():
    import matplotlib.pyplot as plt
    import numpy as np

    x = np.linspace(0, 2 * np.pi, 200)
    fig, ax = plt.subplots(figsize=(6, 3))
    ax.plot(x, np.sin(x), label="sin(x)")
    ax.plot(x, np.cos(x), label="cos(x)")
    ax.set_title("Trigonometric functions")
    ax.legend(loc="upper right")
    ax.set_xlabel("radians")
    ax.set_ylabel("value")
    return fig, (lambda fig=fig: plt.close(fig))


def build_altair_chart():
    import altair as alt
    import pandas as pd

    df = pd.DataFrame(sales_records)
    chart = (
        alt.Chart(df)
        .mark_bar()
        .encode(x="region", y="revenue", color="quarter", column="quarter")
        .properties(width=120, height=200, title="Revenue by region and quarter")
    )
    return chart


def build_seaborn_plot():
    import matplotlib.pyplot as plt
    import seaborn as sns

    penguins = sns.load_dataset("penguins").dropna()
    ax = sns.scatterplot(
        data=penguins,
        x="bill_length_mm",
        y="bill_depth_mm",
        hue="species",
        s=60,
    )
    ax.set_title("Penguin bill measurements")
    return ax, (lambda ax=ax: plt.close(ax.figure))


add_example("NumPy array", build_numpy_array, examples)
add_example("pandas DataFrame", build_pandas_dataframe, examples)
add_example("polars DataFrame", build_polars_dataframe, examples)
add_example("matplotlib Figure", build_matplotlib_figure, examples)
add_example("Altair Chart", build_altair_chart, examples)
add_example("Seaborn scatterplot Axes", build_seaborn_plot, examples)


In [ ]:
for label, obj, cleanup in examples:
    inspect_obj(label, obj, explain=False)
    if cleanup:
        cleanup()


In [ ]:
import vibe_widgets as vw

In [ ]:
vw.load("rollercoaster.vw")